In [13]:
import pandas as pd # 데이터 분석과 처리를 위한 pandas 라이브러리 활용
from tabulate import tabulate  # 데이터프레임을 표 형태로 보기 위해 tabulate 라이브러리 활용

class IndustryDataAnalyzer: # IndustryDataAnalyzer라는 클래스 정의
    def __init__(self, file_paths): # 생성자: 파일 경로를 받아 데이터 로드
        self.file_paths = file_paths # 파일 경로 딕셔너리를 할당
        self.data_frames = {} # 각 산업군에 해당하는 데이터프레임을 저장할 딕셔너리를 초기화
        self.all_companies = {} # 모든 기업 이름을 저장하는 딕셔너리를 초기화
        self.load_data() # load_data

    def load_data(self): # 엑셀 파일 데이터를 읽어와 데이터프레임으로 변환하는 함수
        for category, path in self.file_paths.items():  # 파일 경로 딕셔너리에서 각 산업군 이름(category)과 파일 경로(path)를 반복
            try: # 오류가 발생할 수도 있는 코드를 실행
                df = pd.read_excel(f"/content/sample_data/{path}", engine="openpyxl") # "/content/sample_data/"의 경로에 있는 액셀 파일을 읽어 df에 할당
                self.data_frames[category] = df # df를 data_frames 딕셔너리에서 key인 catecory에 할당
                if not df.empty: # 만약 df가 비어있지 않다면
                    self.all_companies.update({company: category for company in df.iloc[:, 0].dropna()}) # 데이터프레임의 첫번째 열 값을 가져와서 .dropna()로 빈 값을 정리하고 기업 이름을 key로 해당 산업군(category)을 value로 하여 all_companies에 추가
            except Exception as e: # try에서 오류가 발생한다면
                print(f"{category} 파일을 불러오는 중 오류 발생: {e}") # 오류 메시지를 출력

    def display_all_companies(self): # 모든 기업 이름을 산업군별로 보기 좋게 나열하는 함수
        print("[전체 기업 목록]") # 전체 기업 목록의 제목을 출력
        industries = {} # 산업군별로 기업을 분류하기 위한 딕셔너리 생성
        for company, category in self.all_companies.items(): # 생성된 딕셔너리 내의 모든 기업 이름과 해당 산업군을 확인
            if category not in industries: # 만약 현재 산업군이 생성된 딕셔너리에 없으면
                industries[category] = [] # 해당 산업군을 키로 하고 빈 리스트를 값으로 추가
            industries[category].append(company) # 해당 산업군의 리스트에 기업 이름 추가
            # value 값이 리스트 형태로 되어 예를 들어 industries = {"IT 및 소프트웨어": ["Google", "Apple", "Samsung"],"제조업": ["Toyota", "Hyundai"]} 이런 형태가 완성됨

        for category, companies in industries.items(): #산업군 별로 분류된 데이터를 확인
            print(f"[{category}]") # 산업군 이름을 출력
            for company in companies: # 해당 산업군에 속한 기업 이름을 확인
                print(f"  - {company}") # 기업 이름을 들여쓰기 형태로 출력

    def display_company_data(self, company_name): # 특정 기업의 데이터를 출력하는 함수
        if company_name in self.all_companies: # 기업 이름에 해당하는 산업군을 가져옴
            category = self.all_companies[company_name]  #company_name(특정 기업)이 self.all_companies에 존재할 경우 해당 기업의 산업군을 category 변수에 저장함
            df = self.data_frames[category] #  data_frames에서 기업 이름에 해당하는 데이터를 선택
            company_data = df[df.iloc[:, 0] == company_name] # 첫번째 열의 기업 이름 중 선택된 기업의 행을 company_data에 할당
            print(f"\n[{category}] {company_name} 데이터") # 선택된 데이터를 산업군, 기업 이름 형태로 출력
            print(tabulate(company_data, headers='keys', tablefmt='grid'))  # 선택된 데이터를 표 형태로 출력
        else:
            print("입력한 기업 이름이 데이터에 없습니다. 다시 확인해주세요.") # 입력한 기업 이름이 데이터에 없는 경우 메시지로 출력

    def display_category_data(self, category): # 선택한 산업군 데이터를 보기 좋게 표시하는 함수
        if category in self.data_frames: # 만약 data_frames에서 선택한 산업군이 있다면
            df = self.data_frames[category] # category에 해당하는 데이터프레임을 df에 할당
            print(f"[{category}] 데이터") # 선택한 산업군 이름을 출력
            print(tabulate(df, headers='keys', tablefmt='grid'))  # 데이터프레임을 tabulate 라이브러리로 표 형식으로 출력
        else:
            print("잘못된 입력입니다. 다시 시도해주세요.") # 오타가 나면 메시지 출력

    def compare_companies(self, category): # 선택한 산업군의 두 회사를 비교하고 몇 대 몇인지 표시하는 함수
        if category in self.data_frames: # 만약 data_frames에서 선택한 산업군이 있다면
            df = self.data_frames[category] # category에 해당하는 데이터프레임을 df에 할당
            print("기업 목록:") # 기업 목록을 출력
            companies = df.iloc[:, 0].dropna().tolist() # 데이터프레임의 첫번째 열 값을 가져와서 .dropna()로 빈 값을 정리하고 .tolist()를 사용해서 리스트로 변환 후 companies에 할당
            for company in companies:
                print(f"  - {company}") # 각 기업 이름 출력

            while True: # 유효한 입력을 받을 때까지 반복
                company_names = input("비교하고 싶은 2개의 기업을 입력해주세요(기업 사이에 띄어쓰기 필수): ").strip().split() # 입력한 기업 이름 두 개를 깔끔히 분리하여 비교 작업에 사용할 수 있게함

                if len(company_names) == 2: # 입력된 기업 이름이 정확히 두 개라면
                    company1, company2 = company_names # company1, company2에 company_names의 두 기업을 각각 할당
                    if company1 in companies and company2 in companies: # company1과 company2가 companies의 리스트에 있다면
                        c1_data = df[df.iloc[:, 0] == company1].iloc[0] # company1의 데이터를 c1_data에 할당
                        c2_data = df[df.iloc[:, 0] == company2].iloc[0] # company2의 데이터를 c2_data에 할당
                        print(f"\n[{category}] {company1}과 {company2} 비교") # 비교 안내 출력 / 예시([제조업] 한온시스템과 삼양패키징 비교)

                        c1_count, c2_count = 0, 0  # 두 기업의 우위를 비교하기 위한 점수 초기화
                        for col in df.columns[1:]: # 데이터프레임에서 기업 이름 열을 제외한 나머지 열 확인
                            c1_value = c1_data[col] # c1_data의 현재 열 값을 c1_value에 할당
                            c2_value = c2_data[col] # c2_data의 현재 열 값을 c2_value에 할당
                            if c1_value > c2_value: # 만약 c1_value이 c2_value 보다 크다면
                                comparison = f"{company1}({c1_value}) > {company2}({c2_value})" # 비교 결과를 comparison에 할당
                                c1_count += 1 # c1_count의 점수 1 증가
                            elif c1_value < c2_value: # 만약 c2_value이 c1_value보다 크다면
                                comparison = f"{company1}({c1_value}) < {company2}({c2_value})" # 비교 결과를 comparison에 할당
                                c2_count += 1 # c2_count의 점수 1 증가
                            else: # 만약 두 값이 같다면
                                comparison = f"{company1}({c1_value}) = {company2}({c2_value})" # 비교 결과를 comparison에 할당
                            print(f"- {col}: {comparison}") # 현재 열의 비교 결과 출력

                        print(f"\n[{category}] 결과: {company1} {c1_count} 대 {company2} {c2_count}") # 최종 결과 출력
                        break # 루프 종료
                    else:
                        print("기업 이름 중 하나 이상이 데이터에 없습니다. 다시 확인해주세요.") # company1과 company2가 companies의 리스트에 없다면 출력
                else:
                    print("두 개의 기업 이름을 정확하게 입력해주세요.") # 입력된 기업 이름이 정확히 두 개가 아니라면 출력
        else:
            print("잘못된 입력입니다. 다시 시도해주세요.") # 만약 data_frames에서 선택한 산업군이 아니라면 출력

    def ask_for_retry(self, action): # 사용자에게 다시 실행할지 물어보는 함수
        while True: # 무한 루프 시작
            retry = input(f"한 번 더 확인하시겠습니까? (예/아니오): ").strip() # 입력을 받아 .strip()로 입력값의 앞 뒤 공백 제거 후 retry에 할당
            if retry == "예": # 만약 retry가 "예"라면
                return True # True 반환하며 실행 종료
            elif retry == "아니오": # 만약 retry가 "아니오"라면
                return False # False 반환하며 실행 종료
            else:
                print("잘못된 입력입니다. '예' 또는 '아니오'를 입력해주세요.") # 오타가 날 시 출력

    def run(self): # 메인 프로그램을 실행하는 함수
        print("기업 데이터 분석 프로그램에 오신 것을 환영합니다!") # 환영 인사 출력

        print("                                                   자료 출처: 잡코리아") # 자료의 출처를 출력
        while True: # 무한 루프 시작(프로그램이 종료될 때까지 실행)
            print("\n메뉴:")
            print("1. 전체 기업 목록 및 기업 데이터 검색")
            print("2. 산업군 선택 및 기업 데이터 보기")
            print("3. 특정 기업 비교")
            print("4. 프로그램 종료")
            # 메인 메뉴 항목 출력
            choice = input("메뉴에 있는 목록 중 선택하고 싶은 목록의 번호를 입력하세요: ").strip() # 번호를 입력 받아서 .strip()로 공백을 제거하고 choice에 할당

            if choice == "1": # 만약 1번을 눌렀다면
                while True: # 1번 메뉴의 루프 시작
                    self.display_all_companies() # 모든 기업 이름을 산업군별로 보기 좋게 나열하는 함수
                    while True:
                        company_name = input("데이터를 확인할 기업 이름을 하나 입력하세요: ").strip() # 원하는 기업을 입력 받고 공백 제거 후 company_name에 할당
                        if company_name in self.all_companies: # 만약 all_companies에 company_name이 존재한다면
                            self.display_company_data(company_name) # 특정 기업의 데이터를 출력하는 함수를 호출
                            break # 루프 종료 / 다음 단계 실행
                        else:
                            print("입력한 기업 이름이 데이터에 없습니다. 다시 입력해주세요.") # 오타가 날 시 출력
                    if not self.ask_for_retry("1번"): # 사용자에게 다시 실행할지 물어보는 함수를 호출 후 False 반환 시 루프 종료
                        break

            elif choice == "2": # 만약 2번을 눌렀다면
                while True:  # 2번 메뉴의 루프 시작
                    print("산업군 목록: IT 및 소프트웨어, 제조업, 에너지 및 환경, 금융 및 컨설팅, 물류 및 유통") # 산업군 목록 출력
                    category = input("산업군 목록 중 하나를 입력하세요: ").strip() # 원하는 산업군을 입력 받고 공백 제거 후 category에 할당
                    if category in self.data_frames: # 만약 data_frames에 category가 존재한다면
                        self.display_category_data(category) # 선택한 산업군 데이터를 보기 좋게 표시하는 함수를 호출
                        if not self.ask_for_retry("2번"): # 사용자에게 다시 실행할지 물어보는 함수를 호출 후 False 반환 시 루프 종료
                            break
                    else:
                        print("잘못된 산업군 입력입니다. 다시 입력해주세요.") # 오타가 날 시 출력

            elif choice == "3": # 만약 3번을 눌렀다면
                while True: # 3번 메뉴의 루프 시작
                    print("산업군 목록: IT 및 소프트웨어, 제조업, 에너지 및 환경, 금융 및 컨설팅, 물류 및 유통") # 산업군 목록 출력
                    category = input("비교할 기업의 산업군을 하나 입력하세요: ").strip() # 원하는 산업군을 입력 받고 공백 제거 후 category에 할당
                    if category in self.data_frames: # 만약 data_frames에 category가 존재한다면
                        self.compare_companies(category) # 선택한 산업군의 두 회사를 비교하고 몇 대 몇인지 표시하는 함수를 호출
                        if not self.ask_for_retry("3번"): # 사용자에게 다시 실행할지 물어보는 함수를 호출 후 False 반환 시 루프 종료
                            break
                    else:
                        print("잘못된 산업군 입력입니다. 다시 시도해주세요.") # 오타가 날 시 출력

            elif choice == "4": # 만약 4번을 눌렀다면
                print("프로그램을 종료합니다. 이용해 주셔서 감사합니다!") # 메인 프로그램 종료 메시지 출력
                return # 프로그램 종료를 위해 함수 반환

            else:
                print("잘못된 입력입니다. 다시 시도해주세요.") # 오타가 날 시 출력

# 파일 경로 설정
file_paths = {
    "IT 및 소프트웨어": "IT 및 소프트웨어.xlsx",
    "제조업": "제조업.xlsx",
    "에너지 및 환경": "에너지 및 환경.xlsx",
    "금융 및 컨설팅": "금융 및 컨설팅.xlsx",
    "물류 및 유통": "물류 및 유통.xlsx",
}

# 클래스 인스턴스 생성 및 실행
analyzer = IndustryDataAnalyzer(file_paths)
analyzer.run()

기업 데이터 분석 프로그램에 오신 것을 환영합니다!
                                                   자료 출처: 잡코리아

메뉴:
1. 전체 기업 목록 및 기업 데이터 검색
2. 산업군 선택 및 기업 데이터 보기
3. 특정 기업 비교
4. 프로그램 종료
메뉴에 있는 목록 중 선택하고 싶은 목록의 번호를 입력하세요: 1
[전체 기업 목록]
[IT 및 소프트웨어]
  - 더존비즈온
  - 삼성SDS
  - LG CNS
  - NHN
  - 인피닉
[제조업]
  - 한국타이어앤테크놀로지
  - 한온시스템
  - LIG넥스원
  - 삼양패키징
  - LS일렉트릭
[에너지 및 환경]
  - 한국전력공사
  - 한화솔루션
  - 그리드위즈
  - 두산중공업
  - GS칼텍스
[금융 및 컨설팅]
  - 삼정KPMG
  - 딜로이트코리아
  - 현대경제연구원
  - 한화투자증권
  - 삼성생명보험
[물류 및 유통]
  - CJ대한통운
  - 쿠팡
  - 현대글로비스
  - 컬리
  - 롯데글로벌로지스
데이터를 확인할 기업 이름을 하나 입력하세요: 쿠팡

[물류 및 유통] 쿠팡 데이터
+----+-------------+--------------------+------------+--------------+------------+--------------+
|    | 기업 이름   | 올해 입사자 초봉   | 복리후생   | 고용안정성   | 조직문화   | 커리어성장   |
+====+=============+====================+============+==============+============+==============+
|  1 | 쿠팡        | 3,286만 원         | 48점       | 57점         | 40점       | 68점         |
+----+-------------+--------------------+------------+--------------+--